# Scraping data from PropertyGuru 
The following libraries are needed to run this file
<ul> 
    <li> selenium - for scraping
    <li> random - for inducing random factor while scraping in order to avoid being blocked
    <li> calendar
    <li> math 
</ul> 
PS : geckodriver must be in the same directory as this file.

In [9]:
from selenium import webdriver
import random
import time
import re
import math
import pandas as pd
import datetime
import calendar

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys

# remove warning
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [11]:
#Create a firefox driver with specified parameter
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference("general.useragent.override", "Mozilla/5.0 (Android 4.4; Mobile; rv:41.0) Gecko/41.0 Firefox/41.0")

firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['binary'] = 'tools/firefox/firefox-bin'
driver = webdriver.Firefox(firefox_profile=firefox_profile, capabilities=firefox_capabilities)

In [12]:
#Loading data
mrt_data = pd.read_csv('scraped_data/available_MRT.csv',index_col=False)

In [ ]:
#Start scraping the required information from PropertyGuru

for j in range(len(mrt_data)):
        mrt = mrt_data.iloc[j].MRT
        print("Getting %s.."%(mrt))
        
        base_url = mrt_data.iloc[j].link
        driver.get(base_url)
        print("Waiting at most 10 seconds")
        time.sleep(random.randint(1,10))
        
        new_base_url = driver.current_url
        
        print(new_base_url)
        target_num = int(driver.find_element_by_xpath('//h1[@class="title search-title"]//span').text.replace(',',''))

        '''
        while base_url[4:] == new_base_url[5:]:
            new_base_url = driver.current_url
            
        while True:
            try:
                target_num = int(driver.find_element_by_xpath('//h1[@class="title search-title"]//span').text)
                break
            except:
                continue
        '''    
        pages = math.ceil(target_num/20)
    
        
        for p in range(1,pages+1):
            url = new_base_url + '/%s'% (p)
            print("URL")
            print(url)
            driver.get(url)
            
            time.sleep(random.randint(5,20))

            chunk_obj=[]
            
            chunk_obj = driver.find_elements_by_xpath("//div[@class='row']")
            chunk = [x.text for x in chunk_obj]

            '''
            while True:
                try:
                    chunk_obj = driver.find_elements_by_xpath("//div[@class='row']")
                    chunk = [x.text for x in chunk_obj]
                    if (len(chunk_obj) == 22):
                        break
                    if (p == pages and len(chunk_obj) == (target_num%20 + 2)):
                        break
                except:
                    continue
            '''    
            
            
            #Cleaning
            chunk = [chunk[i].split('\n') for i in range(len(chunk))]
            
            del chunk[0]
            del chunk[-1]
            
            print(len(chunk),' records found.')
            
            print("Tidying record...")
            
            chunk = [chunk[i][1:] if len(chunk[i])>7 else chunk[i] for i in range(len(chunk))]
            #Remove records with size less than 7
            chunk = [chunk[i]  for i in range(len(chunk)) if len(chunk[i])>=7]
            
            data = pd.DataFrame(columns=['name','details','size','address','available time','bed','bath','price','nearest MRT','distance'])#'owner','posted on'])
            
            
            data['name'] = [chunk[i][0] for i in range(len(chunk))]
            data['details'] = [chunk[i][1] for i in range(len(chunk))]
            data['address'] = [chunk[i][2] for i in range(len(chunk))]
            
            temp = [chunk[i][3].split('·') for i in range(len(chunk))]
            data['size'] = [temp[i][0] for i in range(len(temp))]
            data['available time'] = [temp[i][1] if len(temp[i])==2 else '' for i in range(len(temp)) ]
            
            #Getting number of beds and bath
            print("Getting number of beds and baths...")
            temp2 = [chunk[i][4].split(' ') for i in range(len(chunk))]
            temp_bed = []
            temp_bath = []
            
            for i in range(len(temp2)):
                if(len(temp2[i]) == 2):
                    temp_bed.append(temp2[i][0])
                    temp_bath.append(temp2[i][1])
                else:
                    try:
                        temp_bed.append(temp2[i][0])
                    except IndexError as e:
                        #print(e)
                        print("At record number %s. 0 had been appended"%i)
                        temp_bed.append(0)
                    try:
                        temp_bath.append(temp2[i][1])
                    except IndexError as e:
                        #print(e)
                        print("At record number %s. 0 had been appended"%i)
                        temp_bath.append(0)
        
            data['bed'] = temp_bed
            data['bath'] = temp_bath         
            data['price'] = [chunk[i][5] for i in range(len(temp))]
            data['distance'] = [re.search('(\d).(\d+) km',x).group() if re.search('(\d).(\d+) km',x) is not None else '' for x in data.address ]
            data['address'] = [x.split(' (')[0] for x in data.address]
            data['nearest MRT ID'] = [mrt for i in range(len(data))]
            
            now = datetime.datetime.now()
            data.to_csv('scraped_data/rental_data_%s_%s%s%s.csv'%(mrt,now.year,calendar.month_name[now.month],now.day),mode='a',header=False)
            print('%s page %s saved'%(mrt,p))

##### \*Note that the last cell was not being runned on Jupyter Notebook as the process took too long (around half day). Besides, sometimes there is also some captcha process to be passed and hence this cell need to be edited a bit and run again, resulting in the existence of rental_data_(MRT station Name)_p2/p3... in scrapped_data folder 